In [1]:
import yfinance as yf
import pandas as pd
import psycopg2

from pandas import set_option
from matplotlib import pyplot as plt
# %matplotlib inline
# %matplotlib inline
# import matplotlib.dates as mpl_dates
#
#
# import plotly.offline as py
# import plotly.graph_objects as go
# import plotly.express as px
# from plotly.subplots import make_subplots

# Data Pipeline

In [14]:

def get_crypto_data(crypto_ticker, start_date, end_date):
    data = pd.DataFrame()  #This is not really nessessary? Remove along with the concatenation that depends on it?

    for each_crypto_ticker in crypto_ticker:
        each_crypto_ticker_index = crypto_ticker.index(each_crypto_ticker) #This is redundant? Remove?
        #Collect data from YFinance
        crypto_data = yf.Ticker(f"{each_crypto_ticker}-USD").history(start=start_date, end=end_date, interval='1d')
        #Store data in Pandas Dataframe
        crypto_dataframe = pd.DataFrame(crypto_data)
        #Create column to uniquely identify each crypto ticker
        crypto_dataframe['crypto_ticker'] = each_crypto_ticker
        #combine with empty data frame created
        data = pd.concat([data, crypto_dataframe])

    # data = data.rename_axis('cryptodate').reset_index()
    data = data.rename_axis('cryptodate')
    return data

In [21]:
#Store data in SQL


def store_in_postgresql(data, table_name):
    try:
        connection = psycopg2.connect(
            user='abelakeni',
            password='unbound365',
            host='34.170.57.79',
            port='5432',
            database='unbound_digital_assets'
        )

        cursor = connection.cursor()

        # Create table if not exists
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (crypto_date DATE, Open FLOAT, High FLOAT, Low FLOAT, Close FLOAT, Volume FLOAT, Dividends FLOAT, Stock_Splits FLOAT, crypto_ticker VARCHAR)"
        cursor.execute(create_table_query)

        # Insert data into the table
        for index, row in data.iterrows():
            sql = f"INSERT INTO {table_name} (crypto_date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, crypto_ticker ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (index.date(), row['Open'], row['High'], row['Low'], row['Close'], row['Volume'], row['Dividends'], row['Stock Splits'], row['crypto_ticker'] ))

        connection.commit()
        print("Data inserted successfully into PostgreSQL database")

    except psycopg2.Error as error:
        print(f"Error: {error}")
    finally:
        if 'connection' in locals() and connection is not None:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")



# Replace 'your_table_name' with the name you want for your table in PostgreSQL
# store_in_postgresql(your_dataframe, 'your_table_name')


In [7]:
crypto_ticker = ['BTC', 'ETH', 'USDT']
start_date = '2015-01-01'
end_date = '2023-11-30'


test_data = get_crypto_data(crypto_ticker, start_date, end_date)
# store_in_postgresql(test_data, 'digital_assets')
test_data

,cryptodate,Open,High,Low,Close,Volume,Dividends,Stock Splits,crypto_ticker
0,2015-01-01,320.434998,320.434998,314.002991,314.248993,8036550,0,0,BTC
1,2015-01-02,314.079010,315.838989,313.565002,315.032013,7860650,0,0,BTC
2,2015-01-03,314.846008,315.149994,281.082001,281.082001,33054400,0,0,BTC
3,2015-01-04,281.145996,287.230011,257.612000,264.195007,55629100,0,0,BTC
4,2015-01-05,265.084015,278.341003,265.084015,274.473999,43962800,0,0,BTC
...,...,...,...,...,...,...,...,...,...
7677,2023-11-26,1.000486,1.000609,0.999623,1.000211,30299804822,0,0,USDT
7678,2023-11-27,1.000214,1.000661,0.999835,0.999980,38568585545,0,0,USDT
7679,2023-11-28,1.000112,1.000668,0.999888,1.000377,37720536626,0,0,USDT
7680,2023-11-29,1.000326,1.000817,0.999889,1.000220,35115076722,0,0,USDT


In [15]:
# No .reset_index() applied to the data
test_data2 = get_crypto_data(crypto_ticker, start_date, end_date)
# store_in_postgresql(test_data, 'digital_assets')
test_data2

,Open,High,Low,Close,Volume,Dividends,Stock Splits,crypto_ticker
cryptodate,,,,,,,,
2015-01-01,320.434998,320.434998,314.002991,314.248993,8036550,0,0,BTC
2015-01-02,314.079010,315.838989,313.565002,315.032013,7860650,0,0,BTC
2015-01-03,314.846008,315.149994,281.082001,281.082001,33054400,0,0,BTC
2015-01-04,281.145996,287.230011,257.612000,264.195007,55629100,0,0,BTC
2015-01-05,265.084015,278.341003,265.084015,274.473999,43962800,0,0,BTC
...,...,...,...,...,...,...,...,...
2023-11-26,1.000486,1.000609,0.999623,1.000211,30299804822,0,0,USDT
2023-11-27,1.000214,1.000661,0.999835,0.999980,38568585545,0,0,USDT
2023-11-28,1.000112,1.000668,0.999888,1.000377,37720536626,0,0,USDT


In [22]:
store_in_postgresql(test_data2, 'digital_assets')


# Error Log: resolved by enabling public access to the database from my home computer

# Error: connection to server at "34.170.57.79", port 5432 failed: Connection timed out (0x0000274C/10060)
# 	Is the server running on that host and accepting TCP/IP connections?

#Error Log2: resolved by putting the complete list of columns before the VALUES (%s...)... the last two column names were missing
# Error: INSERT has more expressions than target columns
# LINE 1: ...314.00299072265625, 314.2489929199219, 8036550, 0, 0, 'BTC')
# sql = f"INSERT INTO {table_name} (crypto_date, Open, High, Low, Close, Volume, Dividends) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"


# ErrorLog3: Resolved by making the column a VARCHAR (could have been TEXT?) 'BTC' column assigned datatype of float when a string was found in the dataframe

# Error: invalid input syntax for type double precision: "BTC"
# LINE 1: ...314.00299072265625, 314.2489929199219, 8036550, 0, 0, 'BTC')
#                                                                  ^
#
# PostgreSQL connection is closed

Data inserted successfully into PostgreSQL database
PostgreSQL connection is closed


In [27]:
#Fetch data from PostgreSQL & convert to dataframe

# My IP Address (86.3.67.247) from WhatsMyIPAddress.com https://whatismyipaddress.com/ 86.3.67.247

def fetch_data_from_sql(table_name):
    try:

        # Connect to Google Cloud SQL PostgreSQL
        connection = psycopg2.connect(
            user='abelakeni',
            password='unbound365',
            host='34.170.57.79',
            port='5432',
            database='unbound_digital_assets'
        )

        cursor = connection.cursor()

        # Execute a query to fetch data from PostgreSQL
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)

        # Fetch all rows from the result
        result = cursor.fetchall()

        # Convert fetched data to a DataFrame
        column_names = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(result, columns=column_names)

        # Display fetched data
        print(df)

    except psycopg2.Error as error:
        print(f"Error: {error}")

    finally:
        if 'connection' in locals() and connection is not None:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

# set_option('display.width', 500)
# Usage example:
fetch_data_from_sql('digital_assets')


     crypto_date        open        high         low       close        volume  dividends  stock_splits crypto_ticker
0     2015-01-01  320.434998  320.434998  314.002991  314.248993  8.036550e+06        0.0           0.0           BTC
1     2015-01-02  314.079010  315.838989  313.565002  315.032013  7.860650e+06        0.0           0.0           BTC
2     2015-01-03  314.846008  315.149994  281.082001  281.082001  3.305440e+07        0.0           0.0           BTC
3     2015-01-04  281.145996  287.230011  257.612000  264.195007  5.562910e+07        0.0           0.0           BTC
4     2015-01-05  265.084015  278.341003  265.084015  274.473999  4.396280e+07        0.0           0.0           BTC
...          ...         ...         ...         ...         ...           ...        ...           ...           ...
7677  2023-11-26    1.000486    1.000609    0.999623    1.000211  3.029980e+10        0.0           0.0          USDT
7678  2023-11-27    1.000214    1.000661    0.999835    

In [6]:
crypto_ticker = ['BTC', 'ETH', 'USDT']
start_date = '2015-01-01'
end_date = '2023-11-30'


test_data = get_crypto_data(crypto_ticker, start_date, end_date)
test_data.tail()

,cryptodate,Open,High,Low,Close,Volume,Dividends,Stock Splits,crypto_ticker
7677,2023-11-26,1.000486,1.000609,0.999623,1.000211,30299804822,0,0,USDT
7678,2023-11-27,1.000214,1.000661,0.999835,0.999980,38568585545,0,0,USDT
7679,2023-11-28,1.000112,1.000668,0.999888,1.000377,37720536626,0,0,USDT
7680,2023-11-29,1.000326,1.000817,0.999889,1.000220,35115076722,0,0,USDT
7681,2023-11-30,1.000236,1.000390,0.999863,1.000156,29247809691,0,0,USDT


In [24]:
# Time functions
import time

start_time = time.time()

# Insert Pandas DataFrame into PostgreSQL using the appropriate method (e.g., psycopg2)
# Your insertion code here

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for insertion: {elapsed_time} seconds")
